In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import itertools
import random

In [ ]:
df = pd.read_csv('features.csv')

# STD scale every features apart from song and bpm : 
features = df.drop(['song', 'bpm'], axis=1)
features = (features - features.mean()) / features.std()
features = features.fillna(0)
df[features.columns] = features

df.head()

In [ ]:
X = df.iloc[:, 2:].values

In [ ]:
# PCA to plot the data in 2D (using plotly)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Keep a column with color that change with bpm increasing (bpm being the first column of X)
X_pca = np.c_[X_pca, df['bpm'].values]

# Plot the data in 2D
fig = px.scatter(x=X_pca[:, 0], y=X_pca[:, 1], color=X_pca[:, 2])
fig.show()


In [ ]:
def create_graph(df):
    G = nx.Graph()
    
    # Add nodes with the BPM attribute
    for index, row in df.iterrows():
        G.add_node(row['song'], bpm=row['bpm'])
    
    # Add edges with a weight attribute if the BPM difference is within the threshold
    for i, row_i in df.iterrows():
        for j, row_j in df.iterrows():
            if i != j: #and abs(row_i['bpm'] - row_j['bpm']) <= threshold:
                # L1 norm : 
                distance =np.abs(row_i[1:] - row_j[1:]).sum()
                G.add_edge(row_i['song'], row_j['song'], weight=distance)
                
    return G

In [ ]:
# 'Pier - Angèle Saiyan (Techno Edit).mp3" bpm is too low, we remove it
df = df[df.song != 'Pier - Angèle Saiyan (Techno Edit).mp3']

# Assuming df is your DataFrame with songs and their BPMs
G = create_graph(df)

# Normalize BPM for color mapping
bpm_normalized = (df.set_index('song')['bpm'] - df['bpm'].min()) / (df['bpm'].max() - df['bpm'].min())

# Calculate node positions
pos = nx.spring_layout(G)

# Node trace
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=10,
        color=[bpm_normalized[node] for node in G.nodes()],
        colorscale='Viridis',
        colorbar=dict(title='BPM'),
        line_width=2,
        opacity=0.8  # Reduced opacity
    ),
    text=[node for node in G.nodes()],  # Hover text
    hoverinfo='text'
)

# Edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Song Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

In [ ]:
def find_shortest_hamiltonian_path(G):
    # Store the shortest path and its length
    shortest_path = None
    shortest_path_length = float('inf')

    # Iterate over all permutations of nodes to check each possible path
    for path in itertools.permutations(G.nodes()):
        current_length = 0
        valid_path = True

        # Calculate the length of the current path
        for i in range(len(path) - 1):
            if G.has_edge(path[i], path[i + 1]):
                current_length += G[path[i]][path[i + 1]]['weight']
            else:
                valid_path = False
                break

        # Update the shortest path if a shorter one is found
        if valid_path and current_length < shortest_path_length:
            shortest_path_length = current_length
            shortest_path = path

        print(f"Checked path: {path}, Length: {current_length}")

    return shortest_path, shortest_path_length


path, length = find_shortest_hamiltonian_path(G)
print(f"Shortest Hamiltonian path: {path}, Length: {length}")